# AutoScale Scaling 정책
- Autoscale에서 관리하고 있는 인스턴스의 숫자를 조절하는 방식
- 4가지 분류
    - 수동 스케일 : 수동으로 직접 인스턴스 숫자를 증감
    - 스케쥴 기반 스케일 : 특정 시점에 인스턴스 숫자를 증감
        - 주로 예측 가능한 시점의 부하 처리 목적으로 활용
    - 동적 스케일 : 특정 기준을 두고 기준치에 따라 인스턴스 숫자를 증감
        - 예 : CPU 사용률 기반, 요청 숫자 기반, 판매량 기반 등
    - 예측 기반 스케일 : 과거 기록의 패턴을 바탕으로 수요량을 예측해서 인스턴스 숫자를 증감

## 수동 스케일
- 말 그대로 수동으로 인스턴스 숫자를 조절하는 방법
    - 주로 개발 환경 혹은 다른 정책을 적용하기 전 사전 테스트 용도로 활용
- 되도록이면 다른 스케일 정책을 비활성화 시킨 후 적용 추천

## 스케쥴 기반 스케일
- 예측 가능한 시점의 변동사항에 대비하여 인스턴스 숫자를 조절하는 방식
    - 예 : 매주 수요일마다 주간 이벤트가 있는 게임
    - 예 : 매일 새벽 특정 시간에 하루동안 모인 데이터를 분석
- 적용 방식
    - 특정 시점을 정해서 해당 시점 or Cron으로 표현하는 반복 시점에
    - 범위 지정 (Min, Max, Desired 셋 중 최소 한가지)
        - 지정한 범위보다 인스턴스가 적다면 Scale Out
        - 지정한 범위보다 인스턴스가 많다면 Scale in

### Cron Expression
- 특정 시간의 주기를 표현하기 위한 표현식
- 1분 단위가 최소 단위
    1. Minutes : Value 0-59, Wildcards ,-*/
    2. Hours : Value 0-23, Wildcards ,-*/
    3. Day-of-month : Value 1-31, Wildcards ,-*?/LW
    4. Month : Value 1-12 or JAN-DEC, Wildcards ,-*/
    5. Day-of-week : Value 1-7 or SUN-SAT, Wildcards ,-*?L#
    6. Year : Value 1970-2199, Wildcards ,-*/
- 예 : 매일 10:00am 마다
    - 분 : 0, 시간 : 10, 날짜 : *, 월 : *, 요일 : ?, 연 : *
- 예 : 매주 월~금 18:00 마다
    - 분 : 0, 시간 : 18, 날짜 : ?, 월 : *, 요일 : MON-FRI, 연 : *
- 예 : 월~금 8시에서 17시 중 5분 마다
    - 분 : 0/5, 시간 : 8-17, 날짜 : ?, 월 : *, 요일 : MON-FRI, 연 : *

## 동적 스케일
- 지표에 반응하여 인스턴스 숫자를 조절하는 방식
    - 주로 Cloudwatch 지표 활용
    - 혹은 자신만의 기준으로 스케일링 정책 조절 가능
- 주로 AutoScale에서 지원하는 추적 조정 정책(Target Tracking Policy)활용
    - 내부적으로 CloudWatch 경보(Alarm)을 생성해서 경보에 반응하여 자동으로 즘감
    - 경보 : CloudWatch 지표에 반응하여 트리거되는 이벤트
    - 지표 증감에 따라 얼마나 민감하게 반응할 것인지 조절 가능
- 필요 시 커스텀 로직을 활용, Autoscale의 증감을 조절 가능

## 예측 기반 스케일
- 사용 패턴의 히스토리를 기반으로 인스턴스 수요량을 예측하여 인스턴스 숫자를 조절하는 방식
    - 주로 반복되는 패턴이 명확한 경우, 혹은 인스턴스 준비가 오래 걸리는 경우 사용
    - 예 : 매일 오후 2시에 레이드 이벤트가 열리는 게임 서비스의 서버
- CloudWatch 지표를 14일간 분석하여 패턴을 만들고 다음 48시간의 사용 패턴을 분석
- 수요량에 따라 증가만 가능
    - 즉, 인스턴스 숫자를 감소시키려면 동적 스케일링 정책 필요
- 주의사항
    - Autoscale 그룹에 다양한 인스턴스 종류가 섞여있을 경우 효율성이 매우 떨어짐
    - 다양한 스케일링 정책이 공존할 경우 효율성이 떨어짐
    - Autoscale Group에 충분한 데이터가 쌓일 때 까지(2주) 기다림 필요

### 실습 - 동적 스케일링 확인
1. 시작 탬플릿 만들기
    - Keypair : X
    - SG : default (인바운드 아웃바운드 전체 허용)
    - 세부 모니터링 활성화
2. Autoscaling Group 만들기
    - 방금 만든 탬플릿 선택
    - 서브넷 모두 선택
    - 용량은 1, 최대용량 3
    - 대상 추적 크기 조정 정책
        - 평균 CPU 사용률 50
    - CloudWatch 내에서 그룹 지표 수집 활성화
    - 태그 Name:demo-asg-policy
3. CloudWatch 설정
    - 모든 경보에 보면 asg의 cpu utilization에 대한 알람을 볼 수 있음
4. 인스턴스의 CPU Utilization 강제 부하
    - 인스턴스 연결 
        - sudo -s
        - dnf install stress -y
        - stress --cpu 1 --timeout 300
    - 모니터링에서 CPU 사용률 확인, Cloudwatch 알람 확인
    - ASG 활동 확인
5. 리소스 정리 : ASG 삭제